# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [5]:
moving_name = 'AtlasV8'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=1)
fixed_all = list_coms(fixed_name, scaling_factor=1)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
#bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

37


In [6]:
fixed_all

{'SC': [9140.45, 2387.99, 5692.5],
 'IC': [10400.0, 2325.0, 5675.0],
 'AP': [12653.24, 5010.46, 5692.13],
 'RtTg': [9503.41, 5321.8, 5692.27],
 'SNR_L': [8440.67, 5167.29, 4031.72],
 'SNR_R': [8440.38, 5167.22, 7353.46],
 'PBG_L': [9402.89, 3844.3, 3551.05],
 'PBG_R': [9400.69, 3846.28, 7833.2699999999995],
 '3N_L': [9102.49, 3794.35, 5520.97],
 '3N_R': [9103.39, 3793.96, 5864.36],
 '4N_L': [9588.57, 3777.83, 5432.47],
 '4N_R': [9586.8, 3774.79, 5952.9],
 'SNC_L': [8354.09, 5124.66, 4293.63],
 'SNC_R': [8355.82, 5122.61, 7092.21],
 'VLL_L': [9464.28, 5175.59, 3889.94],
 'VLL_R': [9465.13, 5173.84, 7495.39],
 '5N_L': [10193.03, 5286.55, 4091.6299999999997],
 '5N_R': [10193.42, 5285.53, 7292.79],
 'LC_L': [10709.59, 4277.99, 4731.98],
 'LC_R': [10709.9, 4278.52, 6652.16],
 'DC_L': [11270.27, 5001.99, 3229.0],
 'DC_R': [11270.93, 5001.97, 8156.389999999999],
 'Sp5C_L': [12832.54, 5896.03, 3980.0],
 'Sp5I_L': [12106.55, 5871.75, 3747.84],
 'Sp5I_R': [12106.52, 5872.03, 7637.71],
 'Sp5O_L':

In [7]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 9.72209073e-01 -4.15690120e-02  1.32902967e-02 -1.34232620e+03]
 [ 1.31329250e-01  1.23381332e+00  7.23778938e-02 -4.42036988e+03]
 [ 5.65136380e-03 -2.11118227e-02  1.14956036e+00 -2.73811629e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 237.06805950085726


In [9]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}-{fixed_name} {structure} {np.round(diff)}')


AtlasV8 SC non trans [10949.  3924.  5199.]
AtlasV8 SC apply trans [9209. 2235. 5682.]
Allen SC [9140. 2388. 5692.]
AtlasV8-Allen SC [  68. -153.  -11.]


In [10]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,AtlasV8,Allen,transformed,difference,sumsquares
Index,,,,,,
12,LRt_L,"[14380.0, 7519.0, 4100.0]","[12344.0, 6991.0, 4393.0]","[12380.0, 7042.0, 4362.0]","[35.0, 52.0, -31.0]",69.8111
6,5N_R,"[11981.0, 6184.0, 6609.0]","[10193.0, 5286.0, 7293.0]","[10136.0, 5262.0, 7261.0]","[-57.0, -24.0, -32.0]",69.8641
25,SNC_R,"[10093.0, 6235.0, 6486.0]","[8356.0, 5123.0, 7092.0]","[8298.0, 5067.0, 7108.0]","[-58.0, -55.0, 16.0]",81.7846
33,PBG_R,"[11244.0, 5053.0, 7099.0]","[9401.0, 3846.0, 7833.0]","[9473.0, 3804.0, 7843.0]","[73.0, -42.0, 10.0]",84.4982
19,5N_L,"[12015.0, 6306.0, 3888.0]","[10193.0, 5287.0, 4092.0]","[10128.0, 5219.0, 4130.0]","[-65.0, -67.0, 39.0]",101.2603
34,IC,"[12075.0, 3840.0, 5165.0]","[10400.0, 2325.0, 5675.0]","[10306.0, 2277.0, 5651.0]","[-94.0, -48.0, -24.0]",108.2127
18,LRt_R,"[14325.0, 7269.0, 6349.0]","[12344.0, 6991.0, 6993.0]","[12366.0, 6889.0, 6952.0]","[22.0, -102.0, -40.0]",111.7658
14,VLL_L,"[11308.0, 6446.0, 3736.0]","[9464.0, 5176.0, 3890.0]","[9433.0, 5288.0, 3949.0]","[-31.0, 113.0, 59.0]",131.0176
5,6N_L,"[12585.0, 6276.0, 4857.0]","[10771.0, 5215.0, 5291.0]","[10697.0, 5327.0, 5248.0]","[-75.0, 113.0, -43.0]",141.5575


In [18]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression


def predict_next_value(sequence):
    if len(sequence) < 2:
        raise ValueError("Need at least two numbers to predict the next value.")
    
    # Reshape data for sklearn
    X = np.array(range(len(sequence))).reshape(-1, 1)
    y = np.array(sequence)
    
    # Fit linear model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict the next value
    next_index = len(sequence)
    next_value = model.predict([[next_index]])
    return next_value[0]


def predict_next_value(sequence, degree=2):
    """
    Predict the next value in a non-linear sequence using polynomial regression.
    
    Args:
        sequence (list): A list of numerical values (non-linear).
        degree (int): Degree of the polynomial regression.

    Returns:
        float: Predicted next value in the sequence.
    """
    if len(sequence) < degree + 1:
        raise ValueError("Sequence too short for chosen polynomial degree.")

    X = np.arange(len(sequence)).reshape(-1, 1)  # [[0], [1], [2], ...]
    y = np.array(sequence)

    # Transform to polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    # Fit the model
    model = LinearRegression()
    model.fit(X_poly, y)

    # Predict the next value
    next_index = np.array([[len(sequence)]])
    next_index_poly = poly.transform(next_index)
    next_value = model.predict(next_index_poly)[0]

    return next_value

In [19]:
sequence = sorted([8.3785, 6.219, 4.1, 2.8, 1.9,1.4,1.0], reverse=False)
# Example usage
next_val = predict_next_value(sequence)
print(sequence)
print("Next predicted value:", next_val)

[1.0, 1.4, 1.9, 2.8, 4.1, 6.219, 8.3785]
Next predicted value: 11.066214285714288


In [20]:
print("Next value prediction:", predict_next_value(sequence, degree=2))

Next value prediction: 11.066214285714288
